### Imports

In [1]:
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Conv2D, LeakyReLU, UpSampling2D, InputLayer, Concatenate, Input, merge, concatenate
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


# Parsing the config

The network config is in `yolov3.cfg`

In [2]:
def parse_config(cfg_path, verbose=False):
    file = open(cfg_path, 'r')
    lines = file.readlines()
        
    # get rid of comments and blank lines and white space
    lines = [x for x in lines if len(x) > 1]
    lines = [x for x in lines if x[0] != '#']
    lines = [x.rstrip().lstrip() for x in lines]

    block = {}
    blocks = []
    
    for line in lines:
        if verbose: print('valuating line: %s' % line)
        
        if line[0] == '[': # new block start
            if len(block) != 0: # if the block inst empty (has data) then reset it
                blocks.append(block)
                block = {}
                
            block['type'] = line[1:-1]
        else:
            key, value = line.split('=')
            key, value = key.rstrip(), value.lstrip()
            block[key] = value

    return blocks

In [3]:
blocks = parse_config('yolov3.cfg')

# The model

(add specific info about the model)

### Questions:
1. Why is padding important in a convolutional layer
2. What is `BatchNormalization` and why is it important
3. What does `bilinear` mean?
4. What is `B X C X H X W` and how is it difforent than other formats
5. What is the difforence between route and shortcut and how are they implemented?
    * see the degradation problem, and an [explaination here](https://www.quora.com/What-are-shortcut-connections-how-do-they-work-and-what-is-their-role-in-the-paper-Deep-Residual-Learning-for-Image-Recognition).

In [14]:
main_modle = Sequential()

all_modules = [Input((608, 608, 3))] # add InputLayer for sequential
module = all_modules[-1]

for index, block in enumerate(blocks[1:]): # block 0 specifies network info
    print('%s, index: %i/%i' % (block['type'], index, len(blocks)))
    
    ### Conv(ish) layers
    if block['type'] == 'convolutional':
        # layer config
        activation = block['activation']
        
        bais = False
        try:
            batch_normalize = int(block['batch_normalize'])
        except:
            batch_normalize = 0
            bais = True
            
        filters = int(block['filters'])
        padding = int(block['pad'])
        kerel_size = int(block['size'])
        strides = int(block['stride'])
        
# not working in keras
#         if padding:
#             pad = (kernel_size - 1) // 2
#         else:
#             pad = 0
        
        # create the conv layer
        module = Conv2D(filters, kerel_size, strides=strides, use_bias=bais)(module)
        
        if batch_normalize:
            module = BatchNormalization()(module)
        
        if activation == 'leaky':
            module = LeakyReLU(0.1)(module)
            
    elif block['type'] == 'upsample':
        stride = int(block['stride'])
        module = UpSampling2D(size=(stride, stride), interpolation='bilinear')(module)

    ### Routs and shortcuts    
    elif block['type'] == 'route': # we need to connect these in the right way
        layers = block['layers'].split(',')
        start = int(layers[0])
        
        try:
           end = int(blcok['layers'][1]) 
        except: # if there is no end
            end = 0
        
        if start > 0:
            start = start - index
        if end > 0:
            end = end - index
        
#         from IPython.core.debugger import Tracer; Tracer()()
        
        print('start: %d, end: %d, all_modules: [%s]' % (start, end, '\n'.join(map(str, all_modules[index + start - 1: index + end]))))
    
        l1 = all_modules[index + start]
        l2 = all_modules[index + end]
        
        if l1.shape[1] == l2.shape[1]: # hack to make sure tensors fit
            module = concatenate([l1, l2])
    
    elif block['type'] == 'shortcut':
        f = int(block['from'])
#         from IPython.core.debugger import Tracer; Tracer()()
        module = all_modules[index + f]

    elif block['type'] == 'yolo': # this needs to also be connected to the network
        mask = block['mask'].split(',')
        mask = [int(x) for x in mask] # convert to ints
        
        anchors = block['anchors'].split(',')
        anchors = [int(a) for a in anchors]
        anchors = [(anchors[i], anchors[i + 1]) for i in range(0, len(anchors), 2)] # break anchors into pairs
        anchors = [anchors[i] for i in mask]
    
    all_modules += [module]

all_modules

convolutional, index: 0/107
convolutional, index: 1/107
convolutional, index: 2/107
convolutional, index: 3/107
shortcut, index: 4/107
convolutional, index: 5/107
convolutional, index: 6/107
convolutional, index: 7/107
shortcut, index: 8/107
convolutional, index: 9/107
convolutional, index: 10/107
shortcut, index: 11/107
convolutional, index: 12/107
convolutional, index: 13/107
convolutional, index: 14/107
shortcut, index: 15/107
convolutional, index: 16/107
convolutional, index: 17/107
shortcut, index: 18/107
convolutional, index: 19/107
convolutional, index: 20/107
shortcut, index: 21/107
convolutional, index: 22/107
convolutional, index: 23/107
shortcut, index: 24/107
convolutional, index: 25/107
convolutional, index: 26/107
shortcut, index: 27/107
convolutional, index: 28/107
convolutional, index: 29/107
shortcut, index: 30/107
convolutional, index: 31/107
convolutional, index: 32/107
shortcut, index: 33/107
convolutional, index: 34/107
convolutional, index: 35/107
shortcut, index:

[<tf.Tensor 'input_11:0' shape=(?, 608, 608, 3) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_584/LeakyRelu:0' shape=(?, 606, 606, 32) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_585/LeakyRelu:0' shape=(?, 302, 302, 64) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_586/LeakyRelu:0' shape=(?, 302, 302, 32) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_587/LeakyRelu:0' shape=(?, 300, 300, 64) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_584/LeakyRelu:0' shape=(?, 606, 606, 32) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_588/LeakyRelu:0' shape=(?, 302, 302, 128) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_589/LeakyRelu:0' shape=(?, 302, 302, 64) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_590/LeakyRelu:0' shape=(?, 300, 300, 128) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_584/LeakyRelu:0' shape=(?, 606, 606, 32) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_591/LeakyRelu:0' shape=(?, 606, 606, 64) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_592/LeakyRelu:0' shape=(?, 604, 604, 128) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_590/Le

In [18]:
main_modle = Model(all_modules[0], module)
main_modle.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 608, 608, 3)       0         
_________________________________________________________________
conv2d_594 (Conv2D)          (None, 606, 606, 32)      864       
_________________________________________________________________
batch_normalization_584 (Bat (None, 606, 606, 32)      128       
_________________________________________________________________
leaky_re_lu_584 (LeakyReLU)  (None, 606, 606, 32)      0         
_________________________________________________________________
conv2d_598 (Conv2D)          (None, 302, 302, 128)     36864     
_________________________________________________________________
batch_normalization_588 (Bat (None, 302, 302, 128)     512       
_________________________________________________________________
leaky_re_lu_588 (LeakyReLU)  (None, 302, 302, 128)     0         
__________